# Compilation Examples with Routing

In [1]:
from pytket import Circuit
from pytket.circuit.display import view_browser as vb
from pytket.extensions.aqt.backends.aqt_multi_zone import AQTMultiZoneBackend
from pytket.extensions.aqt.multi_zone_architecture.named_architectures import (
    four_zones_in_a_line,
    racetrack,
    grid7
)
from pytket.extensions.aqt.multi_zone_architecture.initial_placement.settings import (
    InitialPlacementSettings,
    InitialPlacementAlg,
)
from pytket.extensions.aqt.multi_zone_architecture.circuit_routing.settings import (
    RoutingSettings,
    RoutingAlg,
)
from pytket.extensions.aqt.multi_zone_architecture.compilation_settings import (
    CompilationSettings,
)
import time

First lets define some circuits

In [2]:
## A simple GHZ circuit
ghz_circuit = Circuit(16)
ghz_circuit.H(0)
for i in range(ghz_circuit.n_qubits - 1):
    ghz_circuit.CX(i, i + 1)
ghz_circuit.measure_all()

## Quantum advantage circuit template
def quantum_advantage_circuit(Lx: int, Ly: int) -> Circuit:
    N = Lx * Ly
    Ntrot = 30
    dt = 0.2
    hx = 2
    J = 1

    def c(x, y):
        if (x - y) % N == Lx or (y - x) % N == Lx:
            return 1
        if x // Lx == y // Lx and x == y - (y % Lx) + (((y % Lx) + 1) % Lx):
            return 1
        if x // Lx == y // Lx and y == x - (x % Lx) + (((x % Lx) + 1) % Lx):
            return 1
        return 0

    coupling_list = []
    for i in range(0, N):
        for j in range(i + 1, N):
            if c(i, j) == 1:
                coupling_list.append([i, j])

    U = Circuit(N)

    for t in range(Ntrot):
        n_trotter_steps = t

        if n_trotter_steps == 0:
            for j in range(N):
                U.Rx(-2 * dt * hx / 2, j)

        for coupling in coupling_list:
            U.ZZPhase(-2 * dt * J, coupling[0], coupling[1])
        for j in range(N):
            U.Rx(-2 * dt * hx, j)

        if n_trotter_steps == Ntrot - 1:
            for j in range(N):
                U.Rx(2 * dt * hx / 2, j)
    return U

## 56 qubit quantum advantage circuit
advantage_circuit_56 = quantum_advantage_circuit(7, 8)

## 30 qubit quantum advantage circuit
advantage_circuit_30 = quantum_advantage_circuit(5, 6)



Now some MultiZone Backends

In [3]:
line_backend = AQTMultiZoneBackend(
    architecture=four_zones_in_a_line, access_token="invalid"
)

racetrack.zone_types[0].max_ions = 5
racetrack_backend = AQTMultiZoneBackend(
    architecture=racetrack, access_token="invalid"
)

grid_backend = AQTMultiZoneBackend(
    architecture=grid7, access_token="invalid"
)



In [4]:
## 30 qubit quantum advantage circuit
def build_qft_circuit(n_qubits: int) -> Circuit:
    circ = Circuit(n_qubits, name="QFT")
    for i in range(n_qubits):
        circ.H(i)
        for j in range(i + 1, n_qubits):
            circ.CU1(1 / 2 ** (j - i), j, i)
    for k in range(0, n_qubits // 2):
        circ.SWAP(k, n_qubits - k - 1)
    return circ

nnn = 6
qft_circ = build_qft_circuit(nnn)
#vb(qft_circ)

And some different compilation settings

In [5]:
graph_init = InitialPlacementSettings(
    algorithm=InitialPlacementAlg.graph_partition,
    zone_free_space=2,
    n_threads=1,
    max_depth=200,
)
order_init = InitialPlacementSettings(
    algorithm=InitialPlacementAlg.qubit_order,
    zone_free_space=2,
    n_threads=1,
    max_depth=200,
)
graph_routing = RoutingSettings(
    algorithm=RoutingAlg.graph_partition,
    debug_level=1,
    n_threads=1,
)
graph_compilation_settings = CompilationSettings(
    pytket_optimisation_level=1,
    initial_placement=graph_init,
    routing=graph_routing,
)

greedy_routing = RoutingSettings(
    algorithm=RoutingAlg.greedy,
)

greedy_compilation_settings = CompilationSettings(
    pytket_optimisation_level=1,
    initial_placement=order_init,
    routing=greedy_routing,
)

In [5]:
ghz_precompiled = line_backend.precompile_circuit(ghz_circuit, graph_compilation_settings)
print("#1qb gates =", ghz_precompiled.n_1qb_gates())
print("#gate depth =", ghz_precompiled.depth())
print("#2qb gates =", ghz_precompiled.n_2qb_gates())
print("#2qb gate depth =", ghz_precompiled.depth_2q())
ghz_graph_routed = line_backend.route_precompiled(ghz_precompiled, graph_compilation_settings)
ghz_greedy_routed = line_backend.route_precompiled(ghz_precompiled, greedy_compilation_settings)
print("Shuttles graph: ", ghz_graph_routed.get_n_shuttles())
print("Shuttles greedy: ", ghz_greedy_routed.get_n_shuttles())

#1qb gates = 59
#gate depth = 46
#2qb gates = 15
#2qb gate depth = 15
INITIAL_DEPTH_LIST [[(0, 1)], [(1, 2)], [(2, 3)], [(3, 4)], [(4, 5)], [(5, 6)], [(6, 7)], [(7, 8)], [(8, 9)], [(9, 10)], [(10, 11)], [(11, 12)], [(12, 13)], [(13, 14)], [(14, 15)]]
BLOCKWEIGHTS:  [4, 6, 6, 4]
QUBITTOPART:  [3, 3, 2, 1, 1, 2, 3, 1, 1, 1, 1, 3, 2, 2, 2, 2]
INITPLACE:  {0: [], 1: [3, 4, 7, 8, 9, 10], 2: [2, 5, 12, 13, 14, 15], 3: [0, 1, 6, 11]}
INITIAL PRACEMENT:  {0: [], 1: [3, 4, 7, 8, 9, 10], 2: [2, 5, 12, 13, 14, 15], 3: [0, 1, 6, 11]}
Depth List:
[(1, 2)]
[(2, 3)]
[(3, 4)]
[(4, 5)]
cut_cost:  2228439
-------
Z0: [] -> [] -- ()
Z1: [3, 4, 7, 8, 9, 10] -> [3, 4, 7, 8, 9, 10] -- ()
Z2: [2, 5, 12, 13, 14, 15] -> [1, 2, 5, 12, 13, 14, 15] -- (+1)
Z3: [0, 1, 6, 11] -> [0, 6, 11] -- (-1)
Depth List:
[(2, 3)]
[(3, 4)]
[(4, 5)]
[(5, 6)]
cut_cost:  2244755
-------
Z0: [] -> [] -- ()
Z1: [3, 4, 7, 8, 9, 10] -> [2, 3, 4, 7, 8, 9, 10] -- (+2)
Z2: [1, 2, 5, 12, 13, 14, 15] -> [1, 5, 12, 13, 14, 15] -- (-2)
Z3: [

In [6]:

adv_precomp = racetrack_backend.precompile_circuit(advantage_circuit_56, graph_compilation_settings)
print("#1qb gates =", adv_precomp.n_1qb_gates())
print("#gate depth =", adv_precomp.depth())
print("#2qb gates =", adv_precomp.n_2qb_gates())
print("#2qb gate depth =", adv_precomp.depth_2q())
adv_graph_routed = racetrack_backend.route_precompiled(adv_precomp, graph_compilation_settings)
adv_greedy_routed = racetrack_backend.route_precompiled(adv_precomp, greedy_compilation_settings)
print("Shuttles graph: ", adv_graph_routed.get_n_shuttles())
print("Shuttles greedy: ", adv_greedy_routed.get_n_shuttles())

#1qb gates = 8700
#gate depth = 2162
#2qb gates = 6720
#2qb gate depth = 1094
BLOCKWEIGHTS:  [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
INITPLACE:  {0: [19, 54, 55], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [38, 45], 11: [39, 46, 47], 12: [40, 41, 48], 13: [35, 42, 43], 14: [36, 37, 44], 15: [25, 31, 32], 16: [26, 30, 33], 17: [27, 28, 34], 18: [22, 23, 29], 19: [14, 20, 21], 20: [17, 18, 24], 21: [15, 16, 53], 22: [9, 10, 49], 23: [6, 7, 8], 24: [0, 1, 2], 25: [3, 4, 5], 26: [50, 51, 52], 27: [11, 12, 13]}
INITIAL PRACEMENT:  {0: [19, 54, 55], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [38, 45], 11: [39, 46, 47], 12: [40, 41, 48], 13: [35, 42, 43], 14: [36, 37, 44], 15: [25, 31, 32], 16: [26, 30, 33], 17: [27, 28, 34], 18: [22, 23, 29], 19: [14, 20, 21], 20: [17, 18, 24], 21: [15, 16, 53], 22: [9, 10, 49], 23: [6, 7, 8], 24: [0, 1, 2], 25: [3, 4, 5], 26: [50, 51, 52], 27: [11, 12, 13]}
Depth Li

In [7]:

adv_precomp = grid_backend.precompile_circuit(advantage_circuit_30, graph_compilation_settings)
print("#1qb gates =", adv_precomp.n_1qb_gates())
print("#gate depth =", adv_precomp.depth())
print("#2qb gates =", adv_precomp.n_2qb_gates())
print("#2qb gate depth =", adv_precomp.depth_2q())
adv_graph_routed = grid_backend.route_precompiled(adv_precomp, graph_compilation_settings)
adv_greedy_routed = grid_backend.route_precompiled(adv_precomp, greedy_compilation_settings)
print("Shuttles graph: ", adv_graph_routed.get_n_shuttles())
print("Shuttles greedy: ", adv_greedy_routed.get_n_shuttles())

#1qb gates = 4654
#gate depth = 1269
#2qb gates = 3600
#2qb gate depth = 640
BLOCKWEIGHTS:  [6, 6, 6, 6, 6, 6, 6]
INITPLACE:  {0: [], 1: [0, 1, 2, 3, 4, 5], 2: [20, 24], 3: [15, 16, 19, 28, 29], 4: [6, 7, 8, 25, 26, 27], 5: [17, 18, 21, 22, 23], 6: [9, 10, 11, 12, 13, 14]}
INITIAL PRACEMENT:  {0: [], 1: [0, 1, 2, 3, 4, 5], 2: [20, 24], 3: [15, 16, 19, 28, 29], 4: [6, 7, 8, 25, 26, 27], 5: [17, 18, 21, 22, 23], 6: [9, 10, 11, 12, 13, 14]}
Depth List:
[(1, 6), (1, 6), (0, 25), (0, 25), (2, 7), (2, 7), (3, 8), (4, 9), (3, 8), (4, 9)]
[(1, 26), (5, 6), (1, 26), (5, 6), (2, 27), (2, 27), (3, 28), (4, 29), (3, 28), (4, 29)]
[(6, 7), (0, 1), (6, 7), (0, 1), (5, 9), (5, 9)]
[(6, 11), (7, 8), (1, 2), (6, 11), (7, 8), (1, 2), (5, 10), (0, 4), (5, 10), (0, 4)]
cut_cost:  9173601
-------
Z0: [] -> [1, 2, 6, 7] -- (+1, +2, +6, +7)
Z1: [0, 1, 2, 3, 4, 5] -> [0, 3, 4, 5, 8, 9, 25] -- (+8, +9, +25, -1, -2)
Z2: [20, 24] -> [20, 24] -- ()
Z3: [15, 16, 19, 28, 29] -> [15, 16, 19, 28, 29] -- ()
Z4: [6, 7,